In [ ]:
import folium
import geopandas as gpd
import geoplot
import pandas as pd
import fiona #;help(fiona.open)
import requests
import os, sys
from pandas.io.json import json_normalize # convert json into dataframe
from shapely import wkt
import pandas as pd

# Geopandas shapefile Europa

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world

In [ ]:
europe = world.query('continent == "Europe"')
europe = europe.drop(18) # without russia
#europe

germany = world.query('name == "Germany"')

# Load PyPSA-Eur data


In [ ]:
path_buses = os.getcwd()+'/entsoegridkit/buses.csv'
path_lines = os.getcwd()+'/entsoegridkit/lines.csv'
path_links = os.getcwd()+'/entsoegridkit/links.csv'
path_transformers = os.getcwd()+'/entsoegridkit/transformers.csv'
path_converters = os.getcwd()+'/entsoegridkit/converters.csv'

In [ ]:
#europe_network 
buses = (pd.read_csv(path_buses, quotechar="'",
                         true_values='t', false_values='f',
                         dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))
            
            
lines = (pd.read_csv(path_lines, quotechar="'", true_values='t', false_values='f',
                         dtype=dict(line_id='str', bus0='str', bus1='str',
                                    underground="bool", under_construction="bool"))
         .set_index('line_id')
         .rename(columns=dict(voltage='v_nom', circuits='num_parallel')))


links = (pd.read_csv(path_links, quotechar="'", true_values='t', false_values='f',
                         dtype=dict(link_id='str', bus0='str', bus1='str', under_construction="bool"))
             .set_index('link_id'))


transformers = (pd.read_csv(path_transformers, quotechar="'",
                                true_values='t', false_values='f',
                                dtype=dict(transformer_id='str', bus0='str', bus1='str'))
                    .set_index('transformer_id'))


converters = (pd.read_csv(path_converters, quotechar="'",
                              true_values='t', false_values='f',
                              dtype=dict(converter_id='str', bus0='str', bus1='str'))
                  .set_index('converter_id'))

# Prepare for plot


In [ ]:
#lines
lines = lines.reset_index()
lines['geometry'] = lines['geometry'].apply(wkt.loads)
gdf_lines = gpd.GeoDataFrame(lines, geometry = "geometry",crs="EPSG:4326")

In [ ]:
#buses
gdf_buses = gpd.GeoDataFrame(buses, geometry=gpd.points_from_xy(buses.x, buses.y))

In [ ]:
#links
links = links.reset_index()
links['geometry'] = links['geometry'].apply(wkt.loads)
gdf_links = gpd.GeoDataFrame(links, geometry = "geometry",crs="EPSG:4326")

In [ ]:
#transformers
transformers = pd.merge(transformers, buses, how="left", left_on=["bus0"],  right_on = ['bus_id'])  #match bus with trafo
gdf_transformers = gpd.GeoDataFrame(transformers, geometry=gpd.points_from_xy(transformers.x, transformers.y))

In [ ]:
#converters
converters = pd.merge(converters, buses, how="left", left_on=["bus0"],  right_on = ['bus_id'])  #match bus with converter
gdf_converters = gpd.GeoDataFrame(converters, geometry=gpd.points_from_xy(converters.x, converters.y))


# EU PyPSA input data plot

In [ ]:
ax = gdf_buses.plot(figsize=(20, 20),alpha = 0.5, markersize = 5,color='orange')                                 
geoplot.polyplot(europe ,edgecolor='grey', ax=ax, alpha = 0.5, linewidth = 1)
gdf_lines.plot(figsize=(20, 20),alpha = 0.7, ax=ax) #blue
gdf_links.plot(figsize=(20, 20),alpha = 0.7, ax=ax, color='green') 
gdf_transformers.plot(figsize=(20, 20),alpha = 0.5, markersize = 10, ax=ax, color='magenta')
gdf_converters.plot(figsize=(20, 20),alpha = 0.5, markersize = 30, ax=ax, color='red')

### ZOOM - Europe
xlim = ([europe.total_bounds[0]+25,  europe.total_bounds[2]+25])
ylim = ([europe.total_bounds[1]+20,  europe.total_bounds[3]-5])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

#plt.savefig('africa_transmission_and substations_110.png')

# Germany PyPSA input data plot


In [ ]:
ax = gdf_buses.plot(figsize=(20, 20),alpha = 0.5, markersize = 30,color='orange')                                 
geoplot.polyplot(europe ,edgecolor='grey', ax=ax, alpha = 0.5, linewidth = 5)
gdf_lines.plot(figsize=(20, 20),alpha = 0.7, ax=ax) #blue
gdf_links.plot(figsize=(20, 20),alpha = 0.7, ax=ax, color='green') 
gdf_transformers.plot(figsize=(20, 20),alpha = 0.5, markersize = 60, ax=ax, color='magenta')
gdf_converters.plot(figsize=(20, 20),alpha = 0.5, markersize = 100, ax=ax, color='red')

### ZOOM - Germany
xlim = ([germany.total_bounds[0],  germany.total_bounds[2]])
ylim = ([germany.total_bounds[1],  germany.total_bounds[3]])
ax.set_xlim(xlim)
ax.set_ylim(ylim)

# Folium test ##Not finished 

Folium allows dynamic mapping with zoom function.
This might be good to explore data in detail.

Challenge is to include multiple components into one 'folium' plot.
I think one must add FeatureGroups to solve that.

https://www.analyticsvidhya.com/blog/2020/06/guide-geospatial-analysis-folium-python/

In [ ]:
m = folium.Map(location=[52.4860, 0.65714427359343], zoom_start=8, tiles='CartoDB positron')

folium.GeoJson(data=gdf_lines).add_to(m)

m